In [17]:
# !pip install litellm==1.26.0
import os
import litellm
import sys

In [19]:
# [OPTIONAL] set env var
api_key = os.environ['OPENAI_API_KEY'] # string, you need your own key and to put at least $5 in the account
#s.environ["COHERE_API_KEY"] = api_keys.cohere_trial # string, you can get for free by logging into cohere and going to sandbox

gpt4_turbo = "gpt-4-1106-preview"
cohere = 'command-nightly'
litellm.drop_params=True # will ignore paramaters you set if they don't belong in a model

In [20]:
# from openai import OpenAI
# client = OpenAI()
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

In [21]:
from litellm import completion


def api_request(
    prompt,
    model="commmand-nightly",
    api_key=None,
    temperature=0.1,
    top_p=1,
    timeout=45,
    num_retries=2,
    max_tokens=None,
    seed=None,
    response_format=None,
):
    # Open AI status: https://status.openai.com/

    messages = [{"content": prompt, "role": "user"}]
    responses = completion(
        model=model,
        messages=messages,
        api_key=api_key,
        temperature=temperature,
        top_p=top_p,
        timeout=timeout,
        num_retries=num_retries,
        max_tokens=max_tokens,
        seed=seed,
        # response_format = response_format
    )
    response = responses.get("choices")[0].get("message").get("content")  # access response for first message
    return response



In [22]:
prompt = """
Classify the document into one of the following symptoms:
Symptoms to choose from: {symptoms}

Output only the most likely symptom with the phrase that indicates the symptom. 

For example, 
Document: {example_1}
Symptom: {symptom_1}
Explanation: {example_1}

Document: {example_2}
Symptom: {symptom_2}
Explanation: {example_2}

Document: {document}
Symptom:
"""

In [23]:
symptoms = ['sustained attention', 'hyperactivity', 'disruptive behavior', 'impulse control', 'reward anticipation', 'agression']
symptoms_str = '; '.join(symptoms)
examples = ["I can't sit still", "I am always looking around"]
symptoms_examples = ['hyperactivity', 'sustained attention']
documents = ["In the past two weeks, I interrupted someone while talking"]
responses = {}
for i, document in enumerate(documents):
    prompt_i = prompt.format(symptoms = symptoms_str, example_1 = examples[0], example_2 = examples[1], symptom_1 = symptoms_examples[0], symptom_2 = symptoms_examples[1], document = document)
    print(prompt_i)
    response = api_request(prompt_i, model=gpt4_turbo)
    print(response)
    responses[i] = response


Classify the document into one of the following symptoms:
Symptoms to choose from: sustained attention; hyperactivity; disruptive behavior; impulse control; reward anticipation; agression

Output only the most likely symptom with the phrase that indicates the symptom. 

For example, 
Document: I can't sit still
Symptom: hyperactivity
Explanation: I can't sit still

Document: I am always looking around
Symptom: sustained attention
Explanation: I am always looking around

Document: In the past two weeks, I interrupted someone while talking
Symptom:

Symptom: impulse control
Explanation: I interrupted someone while talking


In [24]:
response.split('\n')

['Symptom: impulse control',
 'Explanation: I interrupted someone while talking']

In [25]:
def generate_prompt(
    construct,
    prompt_name=None,
    prompt="default",
    domain=None,
    definition=None,
    examples=None,
    output_format="default",
    remove_parentheses_definition=True,
):
    if output_format == "default":
        output_format = (
            "Each token should be separated by a semicolon. Do not return duplicate tokens. Do not provide any"
            " explanation or additional text beyond the tokens."
        )
    # removed: Order them by how similar they are to {construct}.
    elif output_format == "json":
        output_format = (
            "Provide tokens in JSON output. Do not return duplicate tokens. Do not provide any explanation or"
            " additional text beyond the tokens."
        )

    # Prompt
    if not isinstance(prompt_name, str):
        # if prompt_name == None:
        prompt_name = construct.replace("_", " ").lower()

    if prompt == "default":
        prompt = "Provide many single words and some short phrases related to"
        if domain:
            domain = f"(in the {domain} domain). "
            prompt = f"""{prompt} {prompt_name} {domain}{output_format}"""
        else:
            prompt = f"""{prompt} {prompt_name}. {output_format}"""
        if definition:
            if remove_parentheses_definition:
                definition = re.sub(r"\(.*?\)", "", definition)  # remove parentheses which is usually citation.
            prompt += f"\nHere is a definition of {prompt_name}: {definition.lower().strip()}"

        if isinstance(examples, list):
            examples = "; ".join(examples)
        if isinstance(examples, str):
            # examples = '; '.join(examples)
            prompt += f"\nHere are some examples (include these in the list): {examples}."

    return prompt

